In [4]:
import bcrypt
from http.server import BaseHTTPRequestHandler, HTTPServer

# Nuestras rutas:
    # - "/": "Hola soy la página de inicio"
    # - "/contact": "Hola soy la página de fromulario de contacto"
    # - "/ruta_no_existente": "Devolvemos un 404 (código de página no encontrada)"

# Listado de código:
    # https://developer.mozilla.org/es/docs/Web/HTTP/Status

import mysql.connector
from mysql.connector import Error

class Database:
    def conexion(self):
        try:
            connex = mysql.connector.connect(
                user = "root",
                password = "",
                host = "localhost",
            )
            if connex.is_connected():
                return connex
        
        except Error as e:
            print("Error en MySQL" + str(e))
        finally:
            print("Conexion a SQL correcta")
        
    def createTable(self, conexionDB):
        try:
            cursor = conexionDB.cursor()
            cursor.execute("drop database if exists test")
            cursor.execute("create databse test")
            cursor.execute("use test")
            cursor.execute("create table datos(id int primary key auto_increment, mensaje varchar(255))")
            cursor.close()
        except Error as e:
            print("Error en MySQL; " + str(e))

    def insertData(self, conexionDB, data):
        try:
            cursor = conexionDB.cursor()
            query = "insert into datos values (default, %s)"
            cursor.execute(query, (data,))
            conexionDB.commit()
            cursor.close()
            conexionDB.close()
        except Error as e:
            print("Error en MySQL; " + str(e))

class Servidor(BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path == "/":
            self.send_response(200)
            self.send_header("Content-type", "text/html;charset=utf-8")
            self.end_headers()
            self.wfile.write(b"Hola soy la pagina de inicio")
            return
            
        if self.path == "/contact":
            self.send_response(200)
            self.send_header("Content-type", "text/html;charset=utf-8")
            self.end_headers()
            self.wfile.write("Hola soy la página de formulario de contacto".encode("utf-8"))
            return
        
        self.send_response(404)
        self.send_header("Content-type", "text/html;charset=utf-8")
        self.end_headers()
        self.wfile.write("Página no encontrada".encode("utf-8"))
        return
    
    def do_POST(self):
        import json
        content_length = int(self.headers["Content-Length"]) # longitud del contenido
        post_data = self.rfile.read(content_length) # lee el contenido
        data = json.loads(post_data.decode("utf-8")) # convertimos el dato que nos viene en json diccionario
        # print(data)

        # Encriptacion
        # data = data["mensaje"]
        mensaje = data["mensaje"].encode("utf-8")

        # # Generating Salt
        salt = bcrypt.gensalt()
        # Hashing Password
        hash_mensaje = bcrypt.hashpw(mensaje, salt)

        # interactuar con la DB

        db = Database()
        conexionDB = db.conexion()
        db.createTable(conexionDB)
        db.insertData(conexionDB, hash_mensaje)

        self.send_response(200)
        self.send_header("Content-type", "text/html")
        self.end_headers()
        self.wfile.write(b"El dato ha sido recibido correctamente")
        return

    
def run():
    print("Iniciando servidor...")
    server_address = ("localhost", 3000)
    server = HTTPServer(server_address, Servidor)
    print("Servidor Localhost escuchando peticiones por el puerto 3000")
    server.serve_forever()

run()

Iniciando servidor...
Servidor Localhost escuchando peticiones por el puerto 3000
Conexion a SQL correcta
Error en MySQL; 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MariaDB server version for the right syntax to use near 'databse test' at line 1
Error en MySQL; 1046 (3D000): No database selected


127.0.0.1 - - [09/Jul/2024 09:37:51] "POST / HTTP/1.1" 200 -


KeyboardInterrupt: 